# Lab9 / simple autoencoder : using Mnist dataset

## Overview

在此lab中，我們使用lab5的的Mnist手寫數字影像資料集，將影像先encode（編碼）成一小於原始input維度的feature，再將這個feature透過decode（解碼）回原始影像維度。

模型的loss為輸出影像（autoencoder後的影像）和原始影像的誤差，最小化這個誤差，使輸出影像接近原始影像。


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing
from scipy import spatial
import random
import os
import cv2
from tensorflow.keras import layers

print(tf.__version__)

## Load data and plot data image

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
n = 10
plt.figure(figsize=(20, 2))
for i in range(1,n):
    ax = plt.subplot(1, n, i)
    plt.imshow(x_train[i])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## Data description

+ x_train的維度: (60000, 28, 28) 60000筆28x28的影像
+ y_train的維度: (60000,) 60000筆x_train影像的正確數字
+ x_test的維度: (10000, 28, 28) 10000筆28x28的影像
+ y_test的維度: (10000,) 10000筆x_test影像的正確數字

In [ ]:
# 看訓練資料跟測試資料的維度。
print("x_train的維度: " + str(x_train.shape)) # (60000, 28, 28)
print("y_train的維度: " + str(y_train.shape))# (60000,)
print("x_test的維度: " + str(x_test.shape)) # (10000, 28, 28)
print("y_test的維度: " + str(y_test.shape)) # (10000,)

此資料集中有x（圖片）與y (圖片對應正確數字)，我們只會用到圖片部分：x_train和x_test，來訓練此autoencoder模型。

x_train為training set，x_test做為訓練模型的validation set。

## Preprocessing
正規化資料數值範圍至 [0, 1] 間

In [ ]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

將維度28x28的影像reshape成長度784的array

In [ ]:
x_train = x_train.reshape(x_train.shape[0], -1) # The "-1" makes reshape flatten the remaining dimensions
x_test = x_test.reshape(x_test.shape[0], -1)

In [ ]:
print(x_train.shape) #(60000, 784)
print(x_test.shape) #(10000, 784)

## Build autoencoder architecture

![autoencoder_schema](https://blog.keras.io/img/ae/autoencoder_schema.jpg)
(source: https://blog.keras.io/building-autoencoders-in-keras.html)

Autoencoder架構中可分為 encoder（編碼器）和 decoder（解碼器）兩部分，它們分別進行壓縮與解壓縮。

encoder將input壓縮成小於原始維度的feature，達到「降維」的作用，再透過decoder重建此壓縮後的feature，得到output。模型的loss為輸出影像和原始影像的誤差，最小化這個誤差，使輸出影像接近原始影像。


In [ ]:
# input為reshape過的array, shape為(784,)
inputs = tf.keras.Input(shape=(784,))

#################### encoder ####################

### 請設計3層的encoder hidden layers，記得使用activation
### [hint] encoder是做「壓縮」，每層output neuron數通常會越來越小
### START CODE HERE ### 




### END CODE HERE ### 

## size of bottleneck: 2
encoder_output = tf.keras.layers.Dense(2)(#output of last encoder layer#)
 
##################### decoder ####################
### 請設計3層的decoder hidden layers，記得使用activation
### [hint] 1. decoder的input為encoder最後的output
###        2. decoder是做「解壓」，每層output neuron數通常會越來越大。此三層的nueron數可以和encoder順序相反
### START CODE HERE ### 


    
### END CODE HERE ### 

## size of decoder output : 與模型的input相同
decoder_output = tf.keras.layers.Dense(784, activation='sigmoid')(#output of last decoder layer#)
 
encoder = tf.keras.Model(inputs=inputs,outputs=encoder_output)
autoencoder = tf.keras.Model(inputs=inputs, outputs=decoder_output)
 
# compile autoencoder
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
encoder.summary()

In [ ]:
autoencoder.summary()

In [ ]:
### autoencoder model training: 
### 請訓練autoendoer模型，以x_train自己為x與y，以x_test作為validation的x與y，
### 並設定參數如epochs,batch_size

### START CODE HERE ### 

### END CODE HERE ### 

## Plot encoder result

在訓練完autoencoder後，我們只先使用前面的encoder部分predict x_test圖片，將其2維結果畫在一平面上，並標上每個點對應的y_test(顏色)。

可以看到encoder的結果就像壓縮一樣將其特徵壓縮成2維（降維），且成功將不同的數字區分開來。

In [ ]:
### 使用encoder層model predict , input 為x_test
test_encode = encoder.predict(x_test)

In [ ]:
test_encode.shape #維度(10000,2) 每筆資料被壓縮成2維特徵

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(test_encode[:, 0], test_encode[:, 1], c=y_test, s=3)
plt.colorbar()
plt.show()

## Image retreival : using encoder feature

了解到encoded結果的2維特徵能成功的分群出數字後，我們也能使用這層的特徵來搜尋相似的影像。

In [ ]:
train_encode = encoder.predict(x_train)

In [ ]:
train_encode.shape

用test的encoded output其中一筆作為query資料，搜尋在train encoded output中與他最像的前5筆資料

In [ ]:
similarity = []

# 從test_encode中取一筆query資料
query_index = 15
query = test_encode[query_index]

# 計算query與全部train_encode資料的相似度
for j in train_encode:
    # cosine similarity = 1 - cosine distance
    cosine = 1 - spatial.distance.cosine(query,j)
    similarity.append(cosine)
similarity = pd.Series(similarity)

#依相似度做降冪排序
sorting = similarity.sort_values(ascending=False)

In [ ]:
#畫出query圖片
plt.figure(figsize=(20, 4))
plt.imshow(x_test[query_index].reshape(28, 28))
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)


In [ ]:
#畫出與query最像的5筆圖片
top_n = 5
for num,i in enumerate(sorting.index[0:top_n]):
    ax = plt.subplot(2, top_n, num + 1)
    plt.imshow(x_train[i].reshape(28, 28))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

## autoencoder 解壓

In [ ]:
result = autoencoder.predict(x_train)

## Plot original input image and autoencoder result

畫出原始input圖片與autoencoder的output圖片

In [ ]:
n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_train[i].reshape(28, 28))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(result[i].reshape(28, 28))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

> 請嘗試調整模型、訓練參數，最小化loss，並查看image retreival結果與原始、解壓圖片的差異